In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/heart-disease-prediction-using-logistic-regression/framingham.csv')
data.head()

In [ ]:
data['TenYearCHD'].value_counts()

In [ ]:
data.isna().sum()

In [ ]:
data.dropna(axis=0,inplace=True)
data['TenYearCHD'].value_counts()

### Fill in the Missing Values : Mode

In [ ]:
data.info()

* Demographic:
* • Sex: male or female(Nominal)
* • Age: Age of the patient;(Continuous - Although the recorded ages have been truncated to whole numbers, the concept of age is continuous)
* Behavioral
* • Current Smoker: whether or not the patient is a current smoker (Nominal)
* • Cigs Per Day: the number of cigarettes that the person smoked on average in one day.(can be considered continuous as one can have any number of cigarettes, even half a cigarette.)
* Medical( history)
* • BP Meds: whether or not the patient was on blood pressure medication (Nominal)
* • Prevalent Stroke: whether or not the patient had previously had a stroke (Nominal)
* • Prevalent Hyp: whether or not the patient was hypertensive (Nominal)
* • Diabetes: whether or not the patient had diabetes (Nominal)
* Medical(current)
* • Tot Chol: total cholesterol level (Continuous)
* • Sys BP: systolic blood pressure (Continuous)
* • Dia BP: diastolic blood pressure (Continuous)
* • BMI: Body Mass Index (Continuous)
* • Heart Rate: heart rate (Continuous - In medical research, variables such as heart rate though in fact discrete, yet are considered continuous because of large number of possible values.)
* • Glucose: glucose level (Continuous)
* Predict variable (desired target)
* • 10 year risk of coronary heart disease CHD (binary: “1”, means “Yes”, “0” means “No”)

# Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
#Gender vs TenYearCHD
x=pd.crosstab(data.male,data.TenYearCHD,margins=True)/data.shape[0]
x

In [ ]:
data.shape[0]

* The dataset of 3656 records has 55.6% percentage data on women and 44.4% data  on men. Out these 7% of women,8.3% of men have a risk of getting coronary heart disease.

* Performed Stratified sampling because of class imbalance.

In [ ]:
from sklearn.model_selection import train_test_split
y=data.pop('TenYearCHD')
X=data
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=1,shuffle=True,stratify=y)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_train =ss.fit_transform(X_train)
X_test =ss.fit_transform(X_test)

In [ ]:

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

lr=LogisticRegression(C=2,solver='liblinear')
lr.fit(X_train,y_train)
lr_y_pred=lr.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,lr_y_pred))
confusion_matrix(y_test,lr_y_pred)

# SVC

In [ ]:
svc=SVC(C=1,kernel='linear')
svc.fit(X_train,y_train)
svc_y_pred=svc.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,svc_y_pred))
confusion_matrix(y_test,svc_y_pred)

The SVC doesn't work for us as it's not giving us any true positives and is probably using high threshold as decision boundary which is expected, compared to logistic regression

# Decision Tree

In [ ]:
dc = DecisionTreeClassifier()
dc.fit(X_train,y_train)
y_pred=dc.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,y_pred))
confusion_matrix(y_test,y_pred)

High Number of misclassifications from the decision decision which makes it worse than the SVC, and Logistic Regression.(FP=120,FN=97)

# Random Forest

In [ ]:
rf=RandomForestClassifier(max_depth=100,criterion='entropy',max_features='log2')
rf.fit(X_train,y_train)
rf_y_pred=rf.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,rf_y_pred))
confusion_matrix(y_test,rf_y_pred)

# XGB 

In [ ]:
xg=  XGBClassifier(n_estimators=50,booster='dart',learning_rate=0.07)
xg.fit(X_train,y_train)
xg_y_pred = xg.predict(X_test)
print("The accuracy is %f" %accuracy_score(y_test,xg_y_pred))
confusion_matrix(y_test,xg_y_pred)

In [ ]:
pd.DataFrame(xg_y_pred).to_csv('submission')

The models are gonna get better predictions with hypertuning for sure, please look at the documentations are try to get the maximum. I hope there was something to learn from my notebook and Look forward to solving more problems! thank you, if you found it useful, please vote!